In [1]:
!python -m pip install neo4j


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import glob

with open("elt_cypher_queries/cleanup_nodes.cypher") as f:
    cleanup_node_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_edges.cypher") as f:
    cleanup_edge_queries = [line for line in f]

with open("elt_cypher_queries/cleanup_constraints.cypher") as f:
    cleanup_constraints_queries = [line for line in f]

node_cypher_files = glob.glob("elt_cypher_queries/nodes/*.cypher")
node_creation_queries = []
for query_file in node_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        node_creation_queries.append(creation_query)

with open("elt_cypher_queries/index_nodes.cypher") as f:
    node_index_queries = [line for line in f]

edge_cypher_files = glob.glob("elt_cypher_queries/edges/*.cypher")
edge_creation_queries = []
for query_file in edge_cypher_files:
    with open(query_file) as f:
        creation_query = f.read()
        edge_creation_queries.append(creation_query)
    

In [3]:
from neo4j import GraphDatabase


def run_my_query(tx):
    result = tx.run(query)
    return list(result)


def execute_query(query, uri, user, password):
    with GraphDatabase.driver(uri, auth=(user, password)) as driver:
        driver.verify_connectivity()
        print("Connection estabilished.")
        with driver.session(database="neo4j") as session:
            result = session.run(query)
        summary = result.consume()
        print(
            "Created {nodes_created} nodes and {relationships_created} relationships in {time} ms.".format(
                nodes_created=summary.counters.nodes_created,
                relationships_created=summary.counters.relationships_created,
                time=summary.result_available_after,
            )
        )


for query in cleanup_constraints_queries + cleanup_edge_queries + cleanup_node_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")

for query in node_creation_queries + node_index_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")

for query in edge_creation_queries:
    print(f"Processing {query}")
    execute_query(query, "bolt://localhost:7687", "neo4j", "pass1234")


with open("elt_cypher_queries/relabel_content.cypher") as f:
    relabel_content_query = f.read()
    print(f"Processing {relabel_content_query}")
    execute_query(relabel_content_query, "bolt://localhost:7687", "neo4j", "pass1234")


Processing DROP CONSTRAINT OMDB_CAST_PK IF EXISTS

Connection estabilished.
Created 0 nodes and 0 relationships in 9 ms.
Processing DROP CONSTRAINT OMDB_CATEGORY_NAME_PK IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 6 ms.
Processing DROP CONSTRAINT OMDB_CATEGORY_NAME IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 4 ms.
Processing DROP CONSTRAINT OMDB_CONTENT_ABSTRACT_PK IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 4 ms.
Processing DROP CONSTRAINT OMDB_CONTENT_ALIAS IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 4 ms.
Processing DROP CONSTRAINT OMDB_CONTENT_CONTENT_LINK_PK IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 4 ms.
Processing DROP CONSTRAINT OMDB_CONTENT_REFERENCE_PK IF EXISTS;

Connection estabilished.
Created 0 nodes and 0 relationships in 4 ms.
Processing DROP CONSTRAINT OMDB_CONTENT_TRAILER_PK IF EXISTS;

Connection estabilishe

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (r) { ... }} {position: line: 1, column: 18, offset: 17} for query: 'MATCH ()-[r]->() CALL { WITH r DETACH DELETE r} IN TRANSACTIONS OF 1000000 ROWS;'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (n) { ... }} {position: line: 1, column: 11, offset: 10} for query: 'MATCH (n) CALL { WITH n DETACH DELETE n} IN TRANSACTIONS OF 1000000 ROWS;'


Connection estabilished.
Created 0 nodes and 0 relationships in 104 ms.
Processing MATCH (n) CALL { WITH n DETACH DELETE n} IN TRANSACTIONS OF 1000000 ROWS;
Connection estabilished.
Created 0 nodes and 0 relationships in 30 ms.
Processing CALL apoc.load.jsonArray('file:///gold_normalized_graph_node_cast.json.gz')
YIELD value
WITH value
CREATE (n:OMDB_CAST {
    cast_id:value.cast_id,
    roles:toStringList(value.roles)
});
Connection estabilished.
Created 412440 nodes and 0 relationships in 2510 ms.
Processing CALL apoc.load.jsonArray('file:///gold_normalized_graph_node_category.json.gz')
YIELD value
WITH value
CREATE (n:OMDB_CATEGORY {
    category_id:toInteger(value.category_id)
});
Connection estabilished.
Created 15275 nodes and 0 relationships in 169 ms.
Processing CALL apoc.load.jsonArray('file:///gold_normalized_graph_node_category_name.json.gz')
YIELD value
WITH value
CREATE (n:OMDB_CATEGORY_NAME {
    name:value.name,
    language_iso_639_1:value.language_iso_639_1
});
Connect